# Google QUEST Q&A Labeling - Inference & Submission

This notebook performs inference using trained DeBERTa-v3-Large models and generates a submission file.
It implements **LightGBM Stacking** (Path A) if OOF predictions are available, otherwise falls back to simple averaging.

**Key Features:**
- Offline compatible (no internet required)
- 5-Fold Ensemble
- Attention Pooling (matching training architecture)
- LightGBM Stacking Post-Processing (with CV)

In [ ]:
import sys
import os

# 1. Setup for Offline Environment
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
sys.modules["tensorflow"] = None # Prevent tensorflow import to avoid protobuf conflicts

import gc
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig
from tqdm.auto import tqdm
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
# ==========================================
# Configuration
# ==========================================
class Config:
    # Paths (Adjust these for Kaggle)
    model_name = "/kaggle/input/deberta-v3-large-offline" # Path to offline tokenizer/config
    model_dir = "/kaggle/input/deberta-v3-large-quest-qa-5fold" # Path to saved weights
    train_csv = "/kaggle/input/google-quest-challenge/train.csv"
    test_csv = "/kaggle/input/google-quest-challenge/test.csv"
    sample_submission = "/kaggle/input/google-quest-challenge/sample_submission.csv"
    output_dir = "/kaggle/working" # Explicit output directory
    
    # Model Config
    max_len = 1024 # Matching training config
    batch_size = 8
    num_workers = 4
    seed = 42
    n_folds = 5
    
    # Target Columns
    target_cols = [
        'question_asker_intent_understanding', 'question_body_critical', 'question_conversational',
        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent',
        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
        'question_type_compare', 'question_type_consequence', 'question_type_definition',
        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written',
        'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
        'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure',
        'answer_type_reason_explanation', 'answer_well_written'
    ]

def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(Config.seed)

In [ ]:
# ==========================================
# Dataset Class
# ==========================================
class QuestDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=512, mode='test'):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.mode = mode
        
        self.titles = df['question_title'].values
        self.bodies = df['question_body'].values
        self.answers = df['answer'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        title = str(self.titles[idx])
        body = str(self.bodies[idx])
        answer = str(self.answers[idx])
        
        # [CLS] title [SEP] body [SEP] answer [SEP]
        q_text = title + " " + self.tokenizer.sep_token + " " + body
        a_text = answer
        
        q_tokens = self.tokenizer.tokenize(q_text)
        a_tokens = self.tokenizer.tokenize(a_text)
        
        # Head+Tail Truncation Logic
        budget = self.max_len - 3
        if len(q_tokens) + len(a_tokens) > budget:
            half = budget // 2
            if len(a_tokens) > half and len(q_tokens) > half:
                a_tokens = a_tokens[:half]
                q_tokens = q_tokens[:budget - len(a_tokens)]
            elif len(a_tokens) <= half:
                q_tokens = q_tokens[:budget - len(a_tokens)]
            else:
                a_tokens = a_tokens[:budget - len(q_tokens)]
                
        ids = [self.tokenizer.cls_token_id] + \
              self.tokenizer.convert_tokens_to_ids(q_tokens) + \
              [self.tokenizer.sep_token_id] + \
              self.tokenizer.convert_tokens_to_ids(a_tokens) + \
              [self.tokenizer.sep_token_id]
              
        mask = [1] * len(ids)
        padding_len = self.max_len - len(ids)
        ids = ids + [self.tokenizer.pad_token_id] * padding_len
        mask = mask + [0] * padding_len
        
        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long)
        }

In [ ]:
# ==========================================
# Model Class (Must match training!)
# ==========================================
class AttentionPooling(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1),
        )

    def forward(self, hidden_states, attention_mask):
        attention_scores = self.attention(hidden_states).squeeze(-1)
        attention_scores = attention_scores.masked_fill(attention_mask == 0, float('-inf'))
        attention_weights = F.softmax(attention_scores, dim=-1)
        pooled = torch.bmm(attention_weights.unsqueeze(1), hidden_states).squeeze(1)
        return pooled

class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None else nn.Parameter(
            torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
        )

    def forward(self, all_hidden_states):
        all_layer_embedding = all_hidden_states[self.layer_start:, :, :, :]
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()
        return weighted_average

class Model(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.config.output_hidden_states = True
        self.bert = AutoModel.from_pretrained(model_name, config=self.config)
        
        # Weighted Layer Pooling
        self.pooler = WeightedLayerPooling(
            self.config.num_hidden_layers, 
            layer_start=4, 
            layer_weights=None
        )
        
        # Attention Pooling
        self.attention_pool = AttentionPooling(self.config.hidden_size)
        
        # Multi-Sample Dropout
        self.dropouts = nn.ModuleList([nn.Dropout(0.1) for _ in range(5)])
        self.linear = nn.Linear(self.config.hidden_size, 30)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        all_hidden_states = torch.stack(outputs.hidden_states)
        
        # Weighted Layer Pooling
        weighted_pooling_embeddings = self.pooler(all_hidden_states)
        
        # Attention Pooling
        pooled_output = self.attention_pool(weighted_pooling_embeddings, attention_mask)
        
        # Multi-Sample Dropout
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                x = self.linear(dropout(pooled_output))
            else:
                x += self.linear(dropout(pooled_output))
        
        x = x / len(self.dropouts)
        return x

In [ ]:
# ==========================================
# Inference Loop
# ==========================================
def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for batch in tqdm(dataloader, desc="Inference"):
        input_ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        
        with torch.no_grad():
            output = model(input_ids, mask)
        
        preds.append(torch.sigmoid(output).cpu().numpy())
        
    return np.concatenate(preds)

def run_inference():
    # Load Data
    test_df = pd.read_csv(Config.test_csv)
    tokenizer = AutoTokenizer.from_pretrained(Config.model_name)
    test_dataset = QuestDataset(test_df, tokenizer, max_len=Config.max_len)
    test_loader = DataLoader(test_dataset, batch_size=Config.batch_size, shuffle=False, num_workers=Config.num_workers)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    fold_preds = []
    
    for fold in range(Config.n_folds):
        print(f"\nRunning Fold {fold}...")
        
        # CORRECTED PATH: best_model/qa_model_fold{fold}_best.bin
        model_path = os.path.join(Config.model_dir, "best_model", f"qa_model_fold{fold}_best.bin")
        
        if not os.path.exists(model_path):
            print(f"⚠️ Model not found: {model_path}. Skipping.")
            continue
            
        model = Model(Config.model_name)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)
        
        preds = inference_fn(model, test_loader, device)
        fold_preds.append(preds)
        
        del model, preds
        torch.cuda.empty_cache()
        gc.collect()
        
    return fold_preds, test_df

In [ ]:
# ==========================================
# Main Execution
# ==========================================
fold_preds, test_df = run_inference()

# Check for OOF predictions for LightGBM Stacking
oof_path = os.path.join(Config.model_dir, "oof_preds.npy")
fold_ids_path = os.path.join(Config.model_dir, "fold_ids.json")
use_stacking = os.path.exists(oof_path) and os.path.exists(Config.train_csv)

if use_stacking:
    print("\n🚀 Found OOF predictions! Running LightGBM Stacking...")
    
    # Load Train Data and OOFs
    train_df = pd.read_csv(Config.train_csv)
    oof_preds = np.load(oof_path) 
    
    # Load Fold IDs if available
    fold_ids = None
    if os.path.exists(fold_ids_path):
        with open(fold_ids_path, "r") as f:
            fold_ids = json.load(f)
        print(f"✅ Loaded Fold IDs for CV.")
    else:
        print(f"⚠️ Fold IDs not found. Will use random split for Early Stopping.")
    
    # Prepare Features
    if len(oof_preds.shape) == 3:
         print(f"⚠️ OOF shape {oof_preds.shape} implies multi-epoch. Using mean of OOFs as features.")
         x_train_all = np.mean(oof_preds, axis=0) # (n_samples, 30)
    else:
        x_train_all = oof_preds
        
    # Test Features
    x_test_all = np.mean(fold_preds, axis=0) # (n_test, 30)
    
    final_preds = np.zeros_like(x_test_all)
    
    lgb_params = {
        "boosting_type": "gbdt",
        "objective": "rmse",
        "learning_rate": 0.1,
        "max_depth": 3,
        "seed": 71,
        "verbose": -1,
        "n_jobs": 4
    }
    
    for i, col in enumerate(Config.target_cols):
        y_train = train_df[col].values
        
        # Use Fold IDs for CV if available
        if fold_ids:
            test_preds_col = np.zeros(len(test_df))
            # fold_ids is list of [train_idx, val_idx]
            for trn_idx, val_idx in fold_ids:
                x_tr, x_val = x_train_all[trn_idx], x_train_all[val_idx]
                y_tr, y_val = y_train[trn_idx], y_train[val_idx]
                
                d_train = lgb.Dataset(x_tr, label=y_tr)
                d_valid = lgb.Dataset(x_val, label=y_val)
                
                model = lgb.train(
                    lgb_params, 
                    d_train, 
                    num_boost_round=1000,
                    valid_sets=[d_valid],
                    callbacks=[lgb.early_stopping(stopping_rounds=30, verbose=False)]
                )
                
                test_preds_col += model.predict(x_test_all) / len(fold_ids)
            
            final_preds[:, i] = test_preds_col
            
        else:
            # Fallback to simple split
            from sklearn.model_selection import train_test_split
            x_tr, x_val, y_tr, y_val = train_test_split(x_train_all, y_train, test_size=0.2, random_state=42)
            
            d_train = lgb.Dataset(x_tr, label=y_tr)
            d_valid = lgb.Dataset(x_val, label=y_val)
            
            model = lgb.train(
                lgb_params, 
                d_train, 
                num_boost_round=1000,
                valid_sets=[d_valid],
                callbacks=[lgb.early_stopping(stopping_rounds=30, verbose=False)]
            )
            
            final_preds[:, i] = model.predict(x_test_all)
        
    print("✅ LightGBM Stacking Completed.")
else:
    print("\nℹ️ OOF predictions not found. Using Simple Averaging.")
    final_preds = np.mean(fold_preds, axis=0)

# Create Submission
sub = pd.DataFrame(final_preds, columns=Config.target_cols)
sub['qa_id'] = test_df['qa_id']
sub = sub[['qa_id'] + Config.target_cols]
output_path = os.path.join(Config.output_dir, "submission.csv")
sub.to_csv(output_path, index=False)
print(f"\n✅ submission.csv saved to {output_path}!")
sub.head()